<a href="https://colab.research.google.com/github/lamsalshrawan/Thesis_RMC_PRnet/blob/main/Utilization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from google.colab import files
import numpy as np

# Explicitly set the 'agg' backend
plt.switch_backend('agg')

# Set global font size to 20
plt.rcParams.update({'font.size': 20})

# Create the DataFrame with chainage data
data = {
    'Months': ['Oct 2022', 'Nov 2022', 'Dec 2022', 'Jan 2023', 'Feb 2023', 'Mar 2023'],
    'Chainage From': [40, 271, 1003, 2053, 2668, 3649],
    'Chainage To': [271, 1003, 2053, 2668, 3649, 4669],
    'Advance (m/months)': [230.84, 731.87, 1049.71, 615.57, 980.72, 1020.12],
    'U (%)': [0.26, 0.45, 0.51, 0.47, 0.52, 0.50],
    'RMR': ['34 - 40', '22 - 48', '31 - 53', '16 - 46', '32 - 55', '16 - 49']
}
df = pd.DataFrame(data)

# Formation data
formation_data = [
    {'Chainage From': 40.5, 'Chainage To': 1260, 'Formation': 'MS1A'},
    {'Chainage From': 1260, 'Chainage To': 2053, 'Formation': 'MS1B'},
    {'Chainage From': 2053, 'Chainage To': 3300, 'Formation': 'MS2A'},
    {'Chainage From': 3300, 'Chainage To': 3649, 'Formation': 'MS2B'},
    {'Chainage From': 3649, 'Chainage To': 3750, 'Formation': 'MS2B'},
    {'Chainage From': 3750, 'Chainage To': 4249, 'Formation': 'US'},
    {'Chainage From': 4249, 'Chainage To': 4669, 'Formation': 'BG'}
]
formation_df = pd.DataFrame(formation_data)

# Define colors for formations
formation_colors = {
    'MS1A': '#ff9a8a', 'MS1B': '#5170d7', 'MS2A': '#bcecac',
    'MS2B': '#c7c10c', 'US': 'crimson', 'BG': '#01a049'
}

# Define heights for subplots
heights = [1, 1, 1, 3]  # Formation, RMR, U, Advance

# Create the multi-panel plot with month-based x-axis
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(12, 12),
                         gridspec_kw={'height_ratios': heights}, dpi=800, sharex=True)

# Month positions and chainages
month_positions = range(len(df['Months']))  # [0, 1, 2, 3, 4, 5] for RMR, U, Advance
month_chainages = df['Chainage From'].tolist() + [df['Chainage To'].iloc[-1]]  # [40.5, 271.0, ..., 4669.0]
chainage_positions = np.linspace(-0.5, 5.5, len(month_chainages))  # 7 positions scaled to -0.5 to 5.5

# --- Top Panel: Formation (using Month grid with interpolated chainages) ---
ax0 = axes[0]
for idx, row in formation_df.iterrows():
    # Interpolate formation chainages within month grid (0 to 5 with padding)
    start_pos = np.interp(row['Chainage From'], month_chainages, chainage_positions)
    end_pos = np.interp(row['Chainage To'], month_chainages, chainage_positions)
    ax0.fill_betweenx([0, 1], start_pos, end_pos,
                      color=formation_colors[row['Formation']], alpha=0.7, label=row['Formation'])

ax0.set_ylim(0, 1)
ax0.set_yticks([])
ax0.set_ylabel('Formation', rotation=90, labelpad=20)  # Rotated 90 degrees
ax0.set_xlim(-0.5, 5.5)  # Match Advance panel limits

# Legend inside plot, 2 rows, 3 columns
formation_patches = [mpatches.Patch(color=formation_colors[formation], label=formation, alpha=0.7)
                     for formation in sorted(formation_df['Formation'].unique())]
ax0.legend(handles=formation_patches, loc='upper center', bbox_to_anchor=(0.5, 0.95),
           fontsize=20, frameon=False, ncol=3)  # 2 rows, 3 columns inside plot

# Set top x-axis for chainages
ax0_top = ax0.twiny()
ax0_top.set_xlim(-0.5, 5.5)  # Match month grid limits
ax0_top.set_xticks(chainage_positions)
ax0_top.set_xticklabels([f'{tick:.0f}' for tick in month_chainages], rotation=45)
ax0_top.set_xlabel('Chainage')

# --- Second Panel: RMR (using Month grid) ---
ax1 = axes[1]
for i, rmr in enumerate(df['RMR']):
    ax1.text(i, 0.5, rmr, ha='center', va='center', fontsize=20, color='#6A1B9A')

ax1.set_ylim(0, 1)
ax1.set_yticks([])
ax1.set_ylabel('RMR', rotation=90, labelpad=20)  # Rotated 90 degrees
ax1.grid(True)

# --- Third Panel: U (%) (using Month grid) ---
ax2 = axes[2]
for i, u in enumerate(df['U (%)']):
    ax2.text(i, 0.5, f'{u:.2f}', ha='center', va='center', fontsize=20, color='#D32F2F')

ax2.set_ylim(0, 1)
ax2.set_yticks([])
ax2.set_ylabel('U (%)', rotation=90, labelpad=20)  # Rotated 90 degrees
ax2.grid(True)

# --- Bottom Panel: Advance (m/month) (using Month grid) ---
ax3 = axes[3]
advance_bars = ax3.bar(month_positions, df['Advance (m/months)'], width=0.8, color='skyblue')  # No label
for i, adv in enumerate(df['Advance (m/months)']):
    ax3.text(i, adv, f'{adv:.2f}', ha='center', va='bottom', fontsize=20)

advance_min = df['Advance (m/months)'].min()
advance_max = df['Advance (m/months)'].max()
advance_range = advance_max - advance_min
advance_buffer = 0.1 * advance_range
ax3.set_ylim(advance_min - advance_buffer, advance_max + advance_buffer)
ax3.set_ylabel('Advance\n(m/month)', rotation=90, labelpad=20)  # Rotated 90 degrees
ax3.grid(True)

# Set bottom x-axis for months
ax3.set_xlim(-0.5, len(df['Months']) - 0.5)  # Add padding for bar alignment
ax3.set_xticks(month_positions)
ax3.set_xticklabels(df['Months'], rotation=45)
ax3.set_xlabel('Months')

# Adjust layout
plt.subplots_adjust(hspace=0.05, top=0.85)  # Adjust top margin for legend

# Save the plot
plt.savefig('four_panel_month_grid_fixed_ticks_plot.jpg', bbox_inches='tight')
files.download('four_panel_month_grid_fixed_ticks_plot.jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from google.colab import files
import numpy as np

# Explicitly set the 'agg' backend
plt.switch_backend('agg')

# Set global font size to 20
plt.rcParams.update({'font.size': 20})

# Create the DataFrame with chainage data
data = {
    'Months': ['Oct 2022', 'Nov 2022', 'Dec 2022', 'Jan 2023', 'Feb 2023', 'Mar 2023'],
    'Chainage From': [40, 271, 1003, 2053, 2668, 3649],
    'Chainage To': [271, 1003, 2053, 2668, 3649, 4669],
    'Advance (m/months)': [230.84, 731.87, 1049.71, 615.57, 980.72, 1020.12],
    'U (%)': [0.26, 0.45, 0.51, 0.47, 0.52, 0.50],
    'RMR': ['34 - 40', '22 - 48', '31 - 53', '16 - 46', '32 - 55', '16 - 49']
}
df = pd.DataFrame(data)

# Formation data
formation_data = [
    {'Chainage From': 40.5, 'Chainage To': 1260, 'Formation': 'MS1A'},
    {'Chainage From': 1260, 'Chainage To': 2053, 'Formation': 'MS1B'},
    {'Chainage From': 2053, 'Chainage To': 3300, 'Formation': 'MS2A'},
    {'Chainage From': 3300, 'Chainage To': 3649, 'Formation': 'MS2B'},
    {'Chainage From': 3649, 'Chainage To': 3750, 'Formation': 'MS2B'},
    {'Chainage From': 3750, 'Chainage To': 4249, 'Formation': 'US'},
    {'Chainage From': 4249, 'Chainage To': 4669, 'Formation': 'BG'}
]
formation_df = pd.DataFrame(formation_data)

# Define colors for formations
formation_colors = {
    'MS1A': '#ff9a8a', 'MS1B': '#5170d7', 'MS2A': '#bcecac',
    'MS2B': '#c7c10c', 'US': 'crimson', 'BG': '#01a049'
}

# Define heights for subplots
heights = [1, 1, 1, 3]  # Formation, RMR, U, Advance

# Create the multi-panel plot with month-based x-axis
fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(12, 12),
                         gridspec_kw={'height_ratios': heights}, dpi=800, sharex=True)

# Month positions and chainages
month_positions = range(len(df['Months']))  # [0, 1, 2, 3, 4, 5] for RMR, U, Advance
month_chainages = df['Chainage From'].tolist() + [df['Chainage To'].iloc[-1]]  # [40, 271, ..., 4669]
chainage_positions = np.linspace(-0.5, 5.5, len(month_chainages))  # 7 positions scaled to -0.5 to 5.5

# --- Top Panel: Formation (using Month grid with interpolated chainages) ---
ax0 = axes[0]
for idx, row in formation_df.iterrows():
    # Interpolate formation chainages within month grid (0 to 5 with padding)
    start_pos = np.interp(row['Chainage From'], month_chainages, chainage_positions)
    end_pos = np.interp(row['Chainage To'], month_chainages, chainage_positions)
    ax0.fill_betweenx([0, 1], start_pos, end_pos,
                      color=formation_colors[row['Formation']], alpha=0.7, label=row['Formation'])

ax0.set_ylim(0, 1)
ax0.set_yticks([])
ax0.set_ylabel('Formation', rotation=90, labelpad=20)  # Rotated 90 degrees
ax0.set_xlim(-0.5, 5.5)  # Match Advance panel limits

# Legend inside plot, 2 rows, 3 columns
formation_patches = [mpatches.Patch(color=formation_colors[formation], label=formation, alpha=0.7)
                     for formation in sorted(formation_df['Formation'].unique())]
ax0.legend(handles=formation_patches, loc='upper center', bbox_to_anchor=(0.5, 0.95),
           fontsize=20, frameon=False, ncol=3)  # 2 rows, 3 columns inside plot

# Set top x-axis for chainages with custom format
ax0_top = ax0.twiny()
ax0_top.set_xlim(-0.5, 5.5)  # Match month grid limits
ax0_top.set_xticks(chainage_positions)
# Custom chainage labels in X+YYY format
chainage_labels = ['0+040', '0+271', '1+003', '2+053', '2+668', '3+649', '4+669']
ax0_top.set_xticklabels(chainage_labels, rotation=45)
ax0_top.set_xlabel('Chainage')

# --- Second Panel: RMR (using Month grid) ---
ax1 = axes[1]
for i, rmr in enumerate(df['RMR']):
    ax1.text(i, 0.5, rmr, ha='center', va='center', fontsize=20, color='#6A1B9A')

ax1.set_ylim(0, 1)
ax1.set_yticks([])
ax1.set_ylabel('RMR', rotation=90, labelpad=20)  # Rotated 90 degrees
ax1.grid(True)

# --- Third Panel: U (%) (using Month grid) ---
ax2 = axes[2]
for i, u in enumerate(df['U (%)']):
    ax2.text(i, 0.5, f'{u:.2f}', ha='center', va='center', fontsize=20, color='#D32F2F')

ax2.set_ylim(0, 1)
ax2.set_yticks([])
ax2.set_ylabel('U', rotation=90, labelpad=20)  # Rotated 90 degrees
ax2.grid(True)

# --- Bottom Panel: Advance (m/month) (using Month grid) ---
ax3 = axes[3]
advance_bars = ax3.bar(month_positions, df['Advance (m/months)'], width=0.8, color='skyblue')  # No label
for i, adv in enumerate(df['Advance (m/months)']):
    ax3.text(i, adv, f'{adv:.2f}', ha='center', va='bottom', fontsize=20)

advance_min = df['Advance (m/months)'].min()
advance_max = df['Advance (m/months)'].max()
advance_range = advance_max - advance_min
advance_buffer = 0.1 * advance_range
ax3.set_ylim(advance_min - advance_buffer, advance_max + advance_buffer)
ax3.set_ylabel('Advance\n(m/month)', rotation=90, labelpad=20)  # Rotated 90 degrees
ax3.grid(True)

# Set bottom x-axis for months
ax3.set_xlim(-0.5, len(df['Months']) - 0.5)  # Add padding for bar alignment
ax3.set_xticks(month_positions)
ax3.set_xticklabels(df['Months'], rotation=45)
ax3.set_xlabel('Months')

# Adjust layout
plt.subplots_adjust(hspace=0.05, top=0.85)  # Adjust top margin for legend

# Save the plot
plt.savefig('four_panel_month_grid_custom_chainage_plot.jpg', bbox_inches='tight')
files.download('four_panel_month_grid_custom_chainage_plot.jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>